#MOUNT AT DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install transformers

#IMPORTS

In [ ]:
import os
import gc
import time
import copy
import csv
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from collections import defaultdict
from transformers import AutoModel, AutoTokenizer, AutoConfig
from tqdm import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#TRAINING DATA

In [ ]:
original_tweets = []
scraped_tweets = []
targets = []

cnt_true = 0
cnt_false = 0


for filename in os.listdir('true15'):
    f = os.path.join('true15', filename)
    cnt_true += 1
    with open(f, mode ='r') as file:
      csvFile = csv.reader(file)
      cnt = 0
      scrap_row = []
      for row in csvFile:
        cnt += 1
        if(cnt==1):
          original_tweets.append(row[0])
        else:
          scrap_row.append(row[0])

      scraped_tweets.append(scrap_row)
      targets.append(1)

for filename in os.listdir('false15'):
    f = os.path.join('false15', filename)
    cnt_false += 1
    with open(f, mode ='r') as file:
      csvFile = csv.reader(file)
      cnt = 0
      scrap_row = []
      for row in csvFile:
        cnt += 1
        if(cnt==1):
          original_tweets.append(row[0])
        else:
          scrap_row.append(row[0])

      scraped_tweets.append(scrap_row)
      targets.append(0)

print("cnt_true: ",cnt_true)
print("cnt_false: ",cnt_false)

cnt_true:  316
cnt_false:  443


#TEST DATA

In [ ]:
import random

indexes = []
for i in range(0,len(original_tweets)):
  indexes.append(i)

test_original_tweets = []
test_scraped_tweets = []
test_targets = []

train_original_tweets = []
train_scraped_tweets = []
train_targets = []

random.shuffle(indexes)
print(indexes)

for i in range(0,100):
  test_original_tweets.append(original_tweets[indexes[i]])
  test_scraped_tweets.append(scraped_tweets[indexes[i]])
  test_targets.append(targets[indexes[i]])

for i in range(100,len(indexes)):
  train_original_tweets.append(original_tweets[indexes[i]])
  train_scraped_tweets.append(scraped_tweets[indexes[i]])
  train_targets.append(targets[indexes[i]])

print((test_targets))
print((train_targets))


[98, 39, 176, 550, 334, 476, 124, 326, 573, 595, 537, 620, 542, 346, 658, 690, 425, 464, 715, 511, 57, 73, 396, 755, 76, 30, 671, 599, 50, 545, 653, 87, 509, 446, 155, 622, 517, 180, 632, 330, 628, 659, 34, 329, 374, 24, 78, 5, 639, 246, 283, 138, 443, 79, 51, 101, 381, 302, 520, 52, 507, 89, 255, 242, 614, 623, 226, 546, 340, 705, 414, 316, 333, 386, 267, 323, 692, 651, 207, 638, 565, 411, 260, 725, 392, 120, 131, 108, 706, 0, 703, 278, 730, 288, 197, 337, 127, 23, 367, 522, 580, 371, 42, 125, 217, 404, 630, 554, 691, 469, 332, 435, 481, 13, 130, 208, 118, 531, 461, 166, 681, 709, 722, 492, 519, 629, 669, 579, 153, 585, 14, 468, 530, 336, 320, 137, 678, 276, 493, 313, 366, 317, 350, 589, 541, 401, 75, 714, 665, 454, 188, 422, 605, 674, 354, 177, 438, 719, 331, 349, 195, 35, 524, 684, 433, 666, 572, 502, 53, 408, 734, 296, 221, 462, 536, 47, 186, 499, 324, 607, 284, 380, 341, 289, 345, 148, 156, 465, 295, 338, 12, 717, 568, 416, 695, 110, 490, 68, 655, 451, 549, 483, 135, 523, 442, 497

#CODE

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, original_tweets, scraped_tweets, targets, tokenizer, max_length=128):
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.tweets = original_tweets
        self.scraped_tweets = scraped_tweets
        self.targets = targets

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, index):
        text = self.tweets[index]
        scraped = self.scraped_tweets[index]
        all_tweets = []
        all_tweets.append(text)
        all_tweets.extend(scraped)
        inputs = self.tokenizer(
                        all_tweets,
                        truncation=True,
                        padding=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        return_tensors='pt'
                    )
        original_tweet = {
            'input_ids': inputs['input_ids'][0],
            'attention_mask': inputs['attention_mask'][0],
        }
        tweets = {
            'input_ids': inputs['input_ids'][1:],
            'attention_mask': inputs['attention_mask'][1:],
        }
        return {
            'original_tweet': original_tweet,
            'tweets': tweets,
            'target': self.targets[index]
        }

In [ ]:
train_ds = FakeNewsDataset(train_original_tweets,
                     train_scraped_tweets,
                     train_targets,
                     tokenizer)
valid_ds = FakeNewsDataset(test_original_tweets,
                     test_scraped_tweets,
                     test_targets,
                     tokenizer)

In [ ]:
class Model(nn.Module):
    def __init__(self, model_name):
        super(Model, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name, config=self.config)
        self.attn_head = nn.MultiheadAttention(embed_dim=self.config.hidden_size, num_heads=8)
        self.qkv = nn.Linear(self.config.hidden_size, 3*self.config.hidden_size)
        self.head = nn.Linear(2*self.config.hidden_size, 1)

    def forward(self, original_tweet, tweets):
        out1 = self.model(input_ids=original_tweet['input_ids'],
                         attention_mask=original_tweet['attention_mask'])
        out2 = self.model(input_ids=tweets['input_ids'].squeeze(0),
                         attention_mask=tweets['attention_mask'].squeeze(0))
        query = self.qkv(out1.pooler_output)[:, :self.config.hidden_size]
        out = self.qkv(out2.pooler_output)
        key, value = out[:, self.config.hidden_size: 2*self.config.hidden_size], out[:, 2*self.config.hidden_size: 3*self.config.hidden_size]
        attn_output, attn_output_weights = self.attn_head(query, key, value)
        concat_vector = torch.concat([out1.pooler_output, attn_output], dim=1)
        output = self.head(concat_vector)
        return output

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
N_ACCUMULATE = 16
BATCH_SIZE = 1
EPOCHS = 5

In [ ]:
def criterion(outputs, labels):
    return nn.BCEWithLogitsLoss()(outputs.view(-1), labels.float().view(-1))

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    model.model.eval()

    dataset_size = 0
    running_loss = 0.0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        original_tweet = {k: v.to(device) for k, v in data['original_tweet'].items()}
        tweets = {k: v.to(device) for k, v in data['tweets'].items()}
        # print(tweets['input_ids'].shape)
        target = data['target'].to(device)

        batch_size = len(original_tweet)

        outputs = model(original_tweet, tweets)
        loss = criterion(outputs, target)
        loss = loss / N_ACCUMULATE

        loss.backward()

        if (step + 1) % N_ACCUMULATE == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()

    return epoch_loss

In [ ]:
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0

    LABELS = []
    PREDS = []

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        original_tweet = {k: v.to(device) for k, v in data['original_tweet'].items()}
        tweets = {k: v.to(device) for k, v in data['tweets'].items()}
        target = data['target'].to(device)

        batch_size = len(original_tweet)

        outputs = model(original_tweet, tweets)
        loss = criterion(outputs, target)

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        PREDS.append(torch.sigmoid(outputs).cpu().detach().numpy())
        LABELS.append(target.cpu().detach().numpy())

        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss)

    LABELS = np.concatenate(LABELS)
    PREDS = np.concatenate(PREDS)
    PREDS = np.where(PREDS > 0.5, 1, 0)
    val_acc = accuracy_score(LABELS, PREDS)

    gc.collect()

    return epoch_loss, val_acc

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_acc = 0
    history = defaultdict(list)

    for epoch in range(1, num_epochs + 1):
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader,
                                           device=device, epoch=epoch)

        val_epoch_loss, val_epoch_acc = valid_one_epoch(model, valid_loader,
                                                       device=device,
                                                       epoch=epoch)

        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid Acc'].append(val_epoch_acc)

        # deep copy the model
        if val_epoch_acc >= best_epoch_acc:
            print(f"Validation Acc Improved ({best_epoch_acc} ---> {val_epoch_acc})")
            best_epoch_acc = val_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "Acc{:.4f}_epoch{:.0f}.bin".format(best_epoch_acc, epoch)
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print("Model Saved")

        print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Acc: {:.4f}".format(best_epoch_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history

In [ ]:
model = Model('bert-base-uncased')
model.to(DEVICE);

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4,
                       weight_decay=1e-6)
scheduler = None

#Data Loader

In [ ]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

In [ ]:
os.listdir()

['true16',
 'false16',
 'false15',
 'true15',
 'Acc0.6800_epoch1.bin',
 'Acc0.6900_epoch2.bin',
 'Acc0.7100_epoch3.bin',
 'Acc0.5300_epoch1.bin',
 'Acc0.6400_epoch1.bin',
 'Tweet Retrieve.ipynb',
 'BTP Model.ipynb']

#RUN MODEL

In [ ]:
model = Model('bert-base-uncased')
model.load_state_dict(torch.load("Acc0.7100_epoch3.bin"))
model.to(DEVICE)
model.eval()
valid_one_epoch(model, valid_loader, DEVICE, epoch=1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 100/100 [00:20<00:00,  4.95it/s, Epoch=1, Valid_Loss=0.425]


(0.4254662098735571, 0.77)

In [ ]:
model, history = run_training(model, optimizer, scheduler,device=DEVICE,num_epochs=EPOCHS)

[INFO] Using GPU: Tesla T4



 64%|██████▍   | 424/659 [03:21<01:51,  2.11it/s, Epoch=1, LR=0.0001, Train_Loss=0.0261]


KeyboardInterrupt: ignored